<a href="https://colab.research.google.com/github/pachterlab/varseek-examples/blob/main/vk_count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [vk count](https://github.com/pachterlab/varseek) demonstration
Perform variant screening on scRNA-seq data with vk count, using a [10x PBMC 1k dataset](https://www.10xgenomics.com/datasets/1-k-pbm-cs-from-a-healthy-donor-v-3-chemistry-3-standard-3-0-0) against the sample mutation database from the vk ref tutorial. 

Written by Joseph Rich.
___

### Install varseek, and import all packages

In [1]:
try:
    import varseek as vk
except ImportError:
    print("varseek not found, installing...")
    !pip install -U -q varseek

In [2]:
import os
import subprocess
import shutil
import anndata as ad
import numpy as np

import varseek as vk

### Define important paths

In [ ]:
# vk count out directory
vk_count_out_dir = os.path.join("data", "varseek_count_out")
reference_dir = os.path.join("data", "reference")

# vk ref out directory and files - downloaded if not already present
vk_ref_out_dir = os.path.join("data", "varseek_ref_out_sample")
vcrs_index = os.path.join(vk_ref_out_dir, "vcrs_index.idx")
vcrs_t2g = os.path.join(vk_ref_out_dir, "vcrs_t2g_filtered.txt")

# fastq directories - fastqs_dir downloaded if not already present, and fastqs_processed_dir created with fastp if not already present
fastqs_dir = os.path.join("data", "pbmc_1k_v3_fastqs")
fastqs_processed_dir = os.path.join(fastqs_dir, "filtered")
technology = "10xv3"
parity = "paired"

# kb count to reference genome directory and files - created if not already present
qc_against_gene_matrix = False
kb_count_reference_genome_dir = os.path.join("data", "kb_count_reference_genome")
reference_genome_index = os.path.join(kb_count_reference_genome_dir, "ensembl_grch37_release93", "index.idx")  # either already exists or will be created
reference_genome_t2g = os.path.join(kb_count_reference_genome_dir, "ensembl_grch37_release93", "t2g.txt")  # either already exists or will be created

# kb ref genome/gtf files - used to create kb index/t2g for reference genome for the step above if reference_genome_index and reference_genome_t2g do not exist
reference_genome_fasta = os.path.join(reference_dir, "ensembl_grch37_release93", "Homo_sapiens.GRCh37.dna.primary_assembly.fa")  # if reference_genome_index/reference_genome_t2g do not exist, then I need to supply the reference genome fasta and gtf
reference_genome_gtf = os.path.join(reference_dir, "ensembl_grch37_release93", "Homo_sapiens.GRCh37.87.gtf")  # if reference_genome_index/reference_genome_t2g do not exist, then I need to supply the reference genome fasta and gtf

# general
threads = 2
k=51

### Download the vk ref index and t2g files if they do not already exist. See vk_ref.ipynb for more details.

In [ ]:
if not os.path.exists(vcrs_index):
    vcrs_index_url = "https://caltech.box.com/shared/static/8693b78lh02fv8qh6wz6keng7cn2n91k.idx"
    vk.utils.download_box_url(vcrs_index_url, output_folder=vk_ref_out_dir, output_file_name="vcrs_index.idx")

if not os.path.exists(vcrs_t2g):
    vcrs_t2g_url = "https://caltech.box.com/shared/static/0svv7xx0mobhfzpiz7f48bjljhs72kcy.txt"
    vk.utils.download_box_url(vcrs_t2g_url, output_folder=vk_ref_out_dir, output_file_name="vcrs_t2g_filtered.txt")

### Download the PBMC fastq dataset

In [5]:
if not os.path.exists(fastqs_dir) or len(os.listdir(fastqs_dir)) == 0:
    !mkdir -p data && \
        cd data && \
        curl -O https://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_1k_v3/pbmc_1k_v3_fastqs.tar && \
        tar -xvf pbmc_1k_v3_fastqs.tar && \
        rm pbmc_1k_v3_fastqs.tar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5292M  100 5292M    0     0  9937k      0  0:09:05  0:09:05 --:--:-- 42.3M0:02:28 34.9M7  0:00:14  0:01:43 47.5M0  44.5M      0  0:01:58  0:00:28  0:01:30 33.0M:00:34  0:01:48 3230k6  0:02:17 4732k19.6M      0  0:04:29  0:01:15  0:03:14 6064k0:05:53  0:01:49  0:04:04 5541k   0  14.6M      0  0:06:01  0:01:53  0:04:08 6124k      0  0:06:39  0:02:14  0:04:25 6699k:04:31 3246k 0  0:08:06  0:03:04  0:05:02 5066k1  0:04:04  0:05:17 3621k    0  9567k      0  0:09:26  0:04:08  0:05:18 4246k      0  0:09:56  0:04:36  0:05:20 5389k   0     0  9034k      0  0:09:59  0:04:39  0:05:20 4746k:06  0:04:47  0:05:19 5318k      0  0:10:26  0:05:13  0:05:13 6873k0     0  8642k      0  0:10:27  0:05:14  0:05:13 4938k5:13 3825kk      0  0:09:14  0:05:44  0:03:30 44.4M  0:03:22 5700k0  9257k      0  0:09:45  0:06:28  0:03:17 4803k      0  0:08:40  

### (Recommended): Process the fastq data - as an example, we will use [fastp](https://github.com/OpenGene/fastp)
Note: fastp was not designed for single-cell data, and as such it requires careful thought to ensure correct processing. The procedure belows works on 10xv3 fastq files, but may need to be modified for other datasets. The function vk.utils.perform_fastp_trimming_and_filtering (called internally by vk.count) handles these nuances. While vk count would handle this step, we call it outside of vk count to demonstrate the process for clarity.

In [8]:
if shutil.which("fastp"):
    if not os.path.exists(fastqs_processed_dir) or len(os.listdir(fastqs_processed_dir)) == 0:
        vk.fastqpp(
            fastqs = fastqs_dir,
            quality_control_fastqs = True,
            technology = technology,
            parity = parity,
            threads = threads,
            cut_front = True,
            cut_tail = True,
            disable_length_filtering = True,
            dont_eval_duplication = True,
            disable_trim_poly_g = True,
            quality_control_fastqs_out_dir = fastqs_processed_dir
        )
        
        # os.makedirs(vk_count_out_dir, exist_ok=True)
        # os.makedirs(fastqs_processed_dir, exist_ok=True)
        # for file_r1, file_r2 in [("pbmc_1k_v3_S1_L001_R1_001.fastq.gz", "pbmc_1k_v3_S1_L001_R2_001.fastq.gz"), ("pbmc_1k_v3_S1_L002_R1_001.fastq.gz", "pbmc_1k_v3_S1_L002_R2_001.fastq.gz")]:
        #     print(f"Processing {file_r1} and {file_r2} with fastp")
        #     file_r1_path = os.path.join(fastqs_dir, file_r1)
        #     file_r2_path = os.path.join(fastqs_dir, file_r2)

        #     file_r1_out_path = os.path.join(fastqs_processed_dir, file_r1)
        #     file_r2_out_path = os.path.join(fastqs_processed_dir, file_r2)

        #     file_r1_out_path_tmp = os.path.join(fastqs_processed_dir, f"tmp_{os.path.basename(file_r1)}")
        #     file_r2_out_path_tmp = os.path.join(fastqs_processed_dir, f"tmp_{os.path.basename(file_r2)}")

        #     # low quality base removal - done separately from edge trimming so that we don't trim bases off of barcode + UMI file
        #     print(f"Low quality base removal for {file_r1} and {file_r2}")
        #     !fastp -i {file_r1_path} -I {file_r2_path} -o {file_r1_out_path_tmp} -O {file_r2_out_path_tmp} --disable_adapter_trimming --qualified_quality_phred 15 --unqualified_percent_limit 40 --average_qual 15 --n_base_limit 10 --disable_length_filtering --dont_eval_duplication --disable_trim_poly_g -h {vk_count_out_dir}/fastp_report1.html -j {vk_count_out_dir}/fastp_report1.json

        #     # edge trimming
        #     print(f"Edge trimming for {file_r2}")
        #     !fastp -i {file_r2_out_path_tmp} -o {file_r2_out_path} --cut_front --cut_tail --cut_window_size 4 --cut_mean_quality 15 --disable_quality_filtering --disable_length_filtering --dont_eval_duplication --disable_trim_poly_g -h {vk_count_out_dir}/fastp_report2.html -j {vk_count_out_dir}/fastp_report2.json --failed_out {vk_count_out_dir}/fastp_failed_tmp.fq

        #     if os.path.getsize(f"{vk_count_out_dir}/fastp_failed_tmp.fq") > 0:
        #         print(f"Removing reads from {file_r1} removed solely from {file_r2} during trimming")
        #         vk.utils.ensure_read_agreement(file_r1_out_path_tmp, file_r2_out_path, f"{vk_count_out_dir}/fastp_failed_tmp.fq", r1_fastq_out_path=file_r1_out_path)
        #     else:
        #         os.rename(file_r1_out_path_tmp, file_r1_out_path)

        #     # break
            
        #     os.remove(f"{vk_count_out_dir}/fastp_failed_tmp.fq")
        #     os.remove(file_r2_out_path_tmp)
else:
    print("fastp is not installed. Skipping fastq pre-processing")
    fastqs_processed_dir = fastqs_dir

01:03:19 - INFO - Removing index files from fastq files list, as they are not utilized in kb count with technology 10xv3
01:03:19 - INFO - Quality controlling fastq files (trimming adaptors, trimming low-quality read edges, filtering low quality reads)
01:03:19 - INFO - Processing data/pbmc_1k_v3_fastqs/pbmc_1k_v3_S1_L001_R1_001.fastq.gz data/pbmc_1k_v3_fastqs/pbmc_1k_v3_S1_L001_R2_001.fastq.gz
Read1 before filtering:
total reads: 33436697
total bases: 936227516
Q20 bases: 914825508(97.714%)
Q30 bases: 875566255(93.5207%)

Read2 before filtering:
total reads: 33436697
total bases: 3042739427
Q20 bases: 2916461207(95.8499%)
Q30 bases: 2748403739(90.3266%)

Read1 after filtering:
total reads: 33073426
total bases: 926055928
Q20 bases: 905632897(97.7946%)
Q30 bases: 867362081(93.662%)

Read2 after filtering:
total reads: 33073426
total bases: 3009681766
Q20 bases: 2900191729(96.3621%)
Q30 bases: 2737346398(90.9514%)

Filtering result:
reads passed filter: 66146852
reads failed due to low 

### (Recommended): Pseudoalign the FASTQ data to the reference genome - helps with adata processing in varseek clean
While vk count would handle this step by providing the reference_genome_index and reference_genome_t2g arguments, we call it outside of vk count to demonstrate the process for clarity. For best quality results, please try to use the reference genome in this step that corresponds to the reference genome assembly and release used in varseek ref.

In [ ]:
kb_count_reference_genome_adata = os.path.join(kb_count_reference_genome_dir, "counts_unfiltered", "adata.h5ad")
if qc_against_gene_matrix and not os.path.exists(kb_count_reference_genome_adata):  # check if kb count was run
    os.makedirs(kb_count_reference_genome_dir, exist_ok=True)
    if not os.path.exists(reference_genome_index) or not os.path.exists(reference_genome_t2g):  # check if kb ref was run
        if not os.path.exists(reference_genome_fasta) or not os.path.exists(reference_genome_gtf):
            reference_genome_out_dir = os.path.dirname(reference_genome_fasta) if reference_genome_fasta else "."
            # using grch37, ensembl 93 to agree with the mutation database
            !gget ref -w dna,gtf -r 93 --out_dir {reference_genome_out_dir} -d human_grch37 && gunzip {reference_genome_fasta}.gz && gunzip {reference_genome_gtf}.gz
        reference_genome_f1 = os.path.join(kb_count_reference_genome_dir, "f1.fa")
        !kb ref -t {threads} -k {k} -i {reference_genome_index} -g {reference_genome_t2g} -f1 {reference_genome_f1} {reference_genome_fasta} {reference_genome_gtf}
    
    !kb count -t {threads} -i {reference_genome_index} -g {reference_genome_t2g} -x {technology} --h5ad --num -o {kb_count_reference_genome_dir} \
        {fastqs_processed_dir}/pbmc_1k_v3_S1_L001_R1_001.fastq.gz {fastqs_processed_dir}/pbmc_1k_v3_S1_L001_R2_001.fastq.gz \
        {fastqs_processed_dir}/pbmc_1k_v3_S1_L002_R1_001.fastq.gz {fastqs_processed_dir}/pbmc_1k_v3_S1_L002_R2_001.fastq.gz

### Run varseek count

This will run the following commands:
- `varseek fastqpp`: Preprocess the fastq files. By default, does nothing.
- `kb count` (variant reference): Perform variant screening on fastq data utilizing kb count's pseudoalignment algorithm. Variant data is stored in an Anndata object as as cell/sample x variant matrix.
- `kb count` ("normal" reference genome) (optional): Perform pseudoalignment of fastq data to the reference genome. Only performed if utilized in the subsequence "varseek clean" step. By default, will occur if the path to the necessary files are not provided as input.
- `varseek clean`: Process the output of kb count. By default, this will threshold variant counts and ensure that there is agreement for each read between the gene of the variant to which the read aligned during the variant reference pseudoalignment and the gene to which the read aligned during the "normal" reference genome pseudoalignment.
- `varseek summarize`: Produces a text file summarizing some high-level insights from the variant screening process.

In [7]:
vk_count_output_dict = vk.count(
    fastqs_processed_dir,
    index=vcrs_index,
    t2g=vcrs_t2g,
    technology=technology,
    out=vk_count_out_dir,
    kb_count_reference_genome_dir=kb_count_reference_genome_dir,
    k=k,
    threads=threads,
    # quality_control_fastqs=True, cut_front=True, cut_tail=True, disable_length_filtering = True, dont_eval_duplication = True, disable_trim_poly_g = True,  # equivalent to the fastp step above
    # reference_genome_index=reference_genome_index, reference_genome_t2g=reference_genome_t2g,  # equivalent to the kb count step above
    qc_against_gene_matrix=qc_against_gene_matrix,
)

22:51:35 - INFO - Removing index files from fastq files list, as they are not utilized in kb count with technology 10XV3
22:51:35 - INFO - Setting length_required to 51 if fastqpp is run
22:51:35 - INFO - Skipping vk fastqpp because there was no use for it
22:51:35 - INFO - Running kb count with command: kb count -t 2 -k 51 -i data/varseek_ref_out_sample/vcrs_index.idx -g data/varseek_ref_out_sample/vcrs_t2g_filtered.txt -x 10XV3 --h5ad -o data/varseek_count_out/kb_count_out_vcrs --overwrite --num --mm --union data/pbmc_1k_v3_fastqs/filtered/pbmc_1k_v3_S1_L001_R1_001.fastq.gz data/pbmc_1k_v3_fastqs/filtered/pbmc_1k_v3_S1_L001_R2_001.fastq.gz data/pbmc_1k_v3_fastqs/filtered/pbmc_1k_v3_S1_L002_R1_001.fastq.gz data/pbmc_1k_v3_fastqs/filtered/pbmc_1k_v3_S1_L002_R2_001.fastq.gz
[2025-07-24 22:51:40,870]    INFO [count] Using index data/varseek_ref_out_sample/vcrs_index.idx to generate BUS file to data/varseek_count_out/kb_count_out_vcrs from
[2025-07-24 22:51:40,871]    INFO [count]        

In [8]:
print(f"Find the processed adata object for further analysis in {vk_count_output_dict['adata_path']}")

Find the processed adata object for further analysis in /Users/joeyrich/Desktop/local/varseek-examples/data/varseek_count_out/adata_cleaned.h5ad


## Load in the anndata object

In [9]:
adata = ad.read_h5ad(vk_count_output_dict["adata_path"])
adata

AnnData object with n_obs × n_vars = 1609 × 150
    var: 'vcrs_id', 'vcrs_header', 'seq_ID', 'mutation', 'nucleotide_positions', 'actual_variant', 'start_variant_position', 'end_variant_position', 'variant_source', 'vcrs_count', 'vcrs_detected', 'number_obs'

## Keep only cells with non-zero counts

In [10]:
adata = adata[:, np.array((adata.X != 0).sum(axis=0)).ravel() > 0]
adata

View of AnnData object with n_obs × n_vars = 1609 × 100
    var: 'vcrs_id', 'vcrs_header', 'seq_ID', 'mutation', 'nucleotide_positions', 'actual_variant', 'start_variant_position', 'end_variant_position', 'variant_source', 'vcrs_count', 'vcrs_detected', 'number_obs'

In [11]:
adata.var.head()

,vcrs_id,vcrs_header,seq_ID,mutation,nucleotide_positions,actual_variant,start_variant_position,end_variant_position,variant_source,vcrs_count,vcrs_detected,number_obs
variant,,,,,,,,,,,,
ENST00000374213:c.184A>G,ENST00000374213:c.184A>G,ENST00000374213:c.184A>G,ENST00000374213,c.184A>G,184,A>G,184,184,transcriptome,2366.0,True,628
ENST00000374213:c.180A>G,ENST00000374213:c.180A>G,ENST00000374213:c.180A>G,ENST00000374213,c.180A>G,180,A>G,180,180,transcriptome,603.0,True,223
ENST00000374222:c.1922G>C,ENST00000374222:c.1922G>C,ENST00000374222:c.1922G>C,ENST00000374222,c.1922G>C,1922,G>C,1922,1922,transcriptome,10.0,True,5
ENST00000374222:c.2004T>C,ENST00000374222:c.2004T>C,ENST00000374222:c.2004T>C,ENST00000374222,c.2004T>C,2004,T>C,2004,2004,transcriptome,4.0,True,2
ENST00000368716:c.413T>A,ENST00000368716:c.413T>A,ENST00000368716:c.413T>A,ENST00000368716,c.413T>A,413,T>A,413,413,transcriptome,4.0,True,2


In [ ]:
# out_df_path = os.path.join("data", "reference", "vk_ref_example", "sample_mutation_database.csv")

# adata = adata[:, ~adata.var_names.str.contains(";")]
# adata.var.head(100)[["seq_ID", "mutation"]].to_csv(out_df_path, index=False)

# adata_empty = ad.read_h5ad(vk_count_output_dict["adata_path"])
# adata_empty = adata_empty[:, np.array((adata_empty.X != 0).sum(axis=0)).ravel() == 0]
# adata_empty = adata_empty[:, ~adata_empty.var_names.str.contains(";")]
# adata_empty.var.head(50)[["seq_ID", "mutation"]].to_csv(out_df_path, index=False, mode="a", header=False)